In [1]:
# Code to evaluate EPMC annotation to the Manual Annotation
# Code to compare results from ML methods and EPMC annotation

# (c) EMBL-EBI, September 2019
#
# Started: 23 Septmember  2019
# Updated: 24 Septmember  2019

_author_ = 'Santosh Tirunagari'

import os
import pandas as pd
import glob
import json
import csv
import sys

import multiprocessing

import numpy as np
import re

from nltk.tokenize import wordpunct_tokenize

import requests
# from pprint import pprint
import pandas as pd

from collections import defaultdict, Counter
import time
from requests.compat import urljoin

from tqdm import tqdm

In [2]:
# Generate train, test and dev pmc ids from the random seed set to 2222
import math
import random
file = '/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/NER/list_pmc_ids.csv'
percentage=0.70
iter = 0

trainPMCids = []
devPMCids = []
testPMCids =[]
try:
    with open(file, 'r',encoding="utf-8") as fin:
        allPMCids = fin.readlines()
except:
    with open('/mnt/droplet'+file, 'r',encoding="utf-8") as fin:
        allPMCids = fin.readlines()    
    
nLines = sum(1 for line in allPMCids)
nTrain = int(nLines*percentage) 
nValid = math.floor((nLines - nTrain)/2)
nTest = nLines - (nTrain+nValid)

deck = list(range(0, nLines))
random.seed(2222) # Please dont change the seed for the reproducibility 
random.shuffle(deck)

train_ids = deck[0:nTrain]
devel_ids = deck[nTrain:nTrain+nValid]
test_ids = deck[nTrain+nValid:nTrain+nValid+nTest]

for each_pmc_id in allPMCids:
    if iter in train_ids:
        trainPMCids.append(each_pmc_id.strip())
    elif iter in devel_ids:
        devPMCids.append(each_pmc_id.strip())
    else:
        testPMCids.append(each_pmc_id.strip())

    iter = iter+1    

In [3]:
def get_Json_through_PMCID(pmcid):
    
    base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
    article_url = urljoin(base_url, "annotationsByArticleIds?articleIds=PMC%3A"+pmcid+"&provider=Europe%20PMC&format=JSON")
    r = requests.get(article_url)
    
    if r.status_code == 200:
        return r
    else:
        return False
    

In [4]:
# result_path = '/mnt/droplet/nfs/gns/literature/Santosh_Tirunagari/EPMC Annotations Dataset/'
result_path = '/nfs/gns/literature/Santosh_Tirunagari/EPMC Annotations Dataset/'

with open(result_path+'EPMC_annotations_.csv','w',  newline='\n') as f1:
    test_writer=csv.writer(f1, delimiter='\t',lineterminator='\n')
    
    for each_test_pmc_id in testPMCids:
        ss = get_Json_through_PMCID(each_test_pmc_id[3:]) # Just the number is needed. SO remove the PMC from the front
        if ss:
            json_results = ss.json()
            pmc_id = json_results[0]['pmcid']
            print(pmc_id)
            for each_annotation in json_results[0]['annotations']:
                exact = each_annotation['prefix']+ each_annotation['exact']+each_annotation['postfix']
                token = each_annotation['tags'][0]['name']
                ner = each_annotation['type']

                row = [pmc_id, exact, token, ner]
                test_writer.writerow(row)
        else:
            continue    

PMC4556948
PMC3651197
PMC5502978
PMC5259676
PMC3613406
PMC3972685
PMC4618948
PMC1762380
PMC4540425
PMC3960246
PMC5770482
PMC4766309
PMC3031208
PMC3316545
PMC5510223
PMC2727484
PMC3174205
PMC2761781
PMC5666160
PMC6037156
PMC1971115
PMC4352028
PMC3844564
PMC4244103
PMC4753424
PMC4697806
PMC3029330
PMC5078810


In [5]:
manual_annot_csv = pd.read_csv('/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/CSV/manual_annot_exacts_180.csv', names=['pmc_id', 'sent_id', 'sentence','ner','relation'], sep ='\t')
manual_annot_csv = manual_annot_csv[manual_annot_csv['ner']!= 'No-Ner']

EPMC_annot_csv = pd.read_csv(result_path+'EPMC_annotations_.csv', names=['pmc_id', 'sentence','token','ner'], sep ='\t')

In [6]:
def sentences_tags(pmc_id, EPMC_annot_csv,manual_annot_csv):
    all_europe_pm_sentences = EPMC_annot_csv[EPMC_annot_csv['pmc_id'] == pmc_id]['sentence'].tolist()
    manual_annotated_sentences = manual_annot_csv[manual_annot_csv['pmc_id'] == pmc_id]['sentence'].tolist()
    
    full_sentences = []
    for each_sentence in tqdm(all_europe_pm_sentences):
        try:
            res = [x for x in manual_annotated_sentences if re.search(re.escape(each_sentence), x)]
            full_sentences.append(res[0])
        except:
            full_sentences.append('None')
    
    new_epmc = EPMC_annot_csv[EPMC_annot_csv['pmc_id'] == pmc_id]
    new_epmc = new_epmc.assign(full_sentence = full_sentences)
    new_epmc['combined'] = new_epmc.apply(lambda x: list([x['token'],x['ner']]),axis=1) 
    sent_tags = new_epmc.groupby('full_sentence')['combined'].apply(list).reset_index(name='tags')
    
    new_man_annot = manual_annot_csv[manual_annot_csv['pmc_id'] == pmc_id]
    new_man_annot = manual_annot_csv[manual_annot_csv['pmc_id'] == pmc_id].reset_index()
    new_man_annot.rename(columns={'sentence':'full_sentence'}, inplace=True)
    epmc_sentence_tags = pd.merge(new_man_annot, sent_tags, on='full_sentence', how='left') 

    
    return epmc_sentence_tags
        
        

In [7]:
from ast import literal_eval

def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll))

    return results


def convert2IOB_dict_manual_annotations(text_data,ner_tags):
    
    tokens = []
    ners = []
    
    split_text = wordpunct_tokenize(text_data)
    # for each word token append 'O'
    arr = ['O']*len(split_text) 
    
    if ner_tags != 'No-Ner':
        ner_tags = literal_eval(ner_tags)
        for each_tag in ner_tags:
            token_list = wordpunct_tokenize(each_tag[2])
            ner_list = wordpunct_tokenize(each_tag[3])

            if(len(token_list) > len(ner_list)):
                ner_list = len(token_list) * ner_list

            for i in range(0,len(ner_list)):
                # The logic here is look for the first B-tag and then append I-tag next
                if(i==0):
                    ner_list[i] = 'B-'+ner_list[i]
                else:
                    ner_list[i] = 'I-'+ner_list[i]

            tokens.append(token_list)
            ners.append(ner_list)

        for i in range(0, len(tokens)):
            spans = find_sub_list(tokens[i], split_text)
            for each_span in spans:
                arr[each_span[0]:each_span[1]] = ners[i]
    
        return zip(split_text, arr)

    else:
        return zip(split_text, arr)
            
            
    
    
def convert2IOB_dict_EPMC_annotations(text_data,ner_tags):
    
    tokens = []
    ners = []
    
    split_text = wordpunct_tokenize(text_data)
    # for each word token append 'O'
    arr = ['O']*len(split_text)
        

    if ner_tags != 'No-Ner':
        for each_tag in ner_tags:
            token_list = wordpunct_tokenize(each_tag[0])
            ner_list = wordpunct_tokenize(each_tag[1])

            if(len(token_list) > len(ner_list)):
                ner_list = len(token_list) * ner_list

            for i in range(0,len(ner_list)):
                # The logic here is look for the first B-tag and then append I-tag next
                if(i==0):
                    ner_list[i] = 'B-'+ner_list[i]
                else:
                    ner_list[i] = 'I-'+ner_list[i]

            tokens.append(token_list)
            ners.append(ner_list)

        for i in range(0, len(tokens)):
            spans = find_sub_list(tokens[i], split_text)
            for each_span in spans:
                arr[each_span[0]:each_span[1]] = ners[i]

        return zip(split_text, arr)
    else:
        return zip(split_text, arr)

In [8]:
from nltk.tokenize import wordpunct_tokenize

result_path = '/nfs/gns/literature/Santosh_Tirunagari/EPMC Annotations Dataset/'

with open(result_path+'EPMC_annotated_test.csv','w',  newline='\n') as f1, open(result_path+'manual_annotated_test.csv','w',  newline='\n') as f2: 
    epmc_writer=csv.writer(f1, delimiter='\t',lineterminator='\n')
    manual_writer=csv.writer(f2, delimiter='\t',lineterminator='\n')
    
    for each_pmc_id in testPMCids:
        print(each_pmc_id)
        ss_ = sentences_tags(each_pmc_id, EPMC_annot_csv,manual_annot_csv)
        ss = ss_.where((pd.notnull(ss_)), 'No-Ner')
        for index, row in tqdm(ss.iterrows(), total=ss.shape[0]):
            tagged_tokens = convert2IOB_dict_EPMC_annotations(row['full_sentence'],row['tags'])
            for each_word in tagged_tokens: # make it to manual annotation format for easy comparison
                epmc_tokens = list(each_word)
                if epmc_tokens[1] == 'B-Diseases':
                    epmc_tokens[1] = 'B-DS'
                elif epmc_tokens[1] == 'I-Diseases':
                    epmc_tokens[1] = 'I-DS'
                elif epmc_tokens[1] == 'B-Organisms':
                    epmc_tokens[1] = 'B-OG'
                elif epmc_tokens[1] == 'I-Organisms':
                    epmc_tokens[1] = 'I-OG'  
                elif epmc_tokens[1] == 'B-Gene_Proteins':
                    epmc_tokens[1] = 'B-GP'   
                elif epmc_tokens[1] == 'I-Gene_Proteins':
                    epmc_tokens[1] = 'I-GP'  
                else:
                    epmc_tokens[1] = 'O'
                epmc_writer.writerow(epmc_tokens)
            epmc_writer.writerow('') 
            manual_tagged_tokens = convert2IOB_dict_manual_annotations(row['full_sentence'],row['ner'])
            for each_word in manual_tagged_tokens:
                manual_writer.writerow(list(each_word))
            manual_writer.writerow('') 

 46%|████▋     | 204/441 [00:00<00:00, 922.18it/s]

PMC4556948


 27%|██▋       | 158/578 [00:00<00:00, 1575.17it/s]

PMC3651197


100%|██████████| 113/113 [00:00<00:00, 2692.93it/s]


...

PMC4753424
PMC4697806


 37%|███▋      | 135/362 [00:00<00:00, 1345.34it/s]

PMC3029330


100%|██████████| 160/160 [00:00<00:00, 707899.41it/s]
0it [00:00, ?it/s]

PMC5078810


In [50]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

true_lbs = pd.read_csv(result_path+'manual_annotated_test.csv', sep='\t', names=['tokens','tags'])
pred_lbs = pd.read_csv(result_path+'EPMC_annotated_test.csv', sep='\t', names=['tokens','tags'])

y_true = true_lbs['tags'].values
y_pred =  pred_lbs['tags'].values


class_labels = sorted([tag for tag in set(y_true) if tag != 'O'], key=lambda name: (name[1:], name[0]))

from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, labels = class_labels))


              precision    recall  f1-score   support

        B-DS       0.74      0.53      0.62      1323
        I-DS       0.84      0.70      0.76       486
        B-GP       0.76      0.52      0.62      3056
        I-GP       0.89      0.41      0.56      1989
        B-OG       0.93      0.69      0.79      1960
        I-OG       0.98      0.51      0.68       968

   micro avg       0.83      0.54      0.66      9782
   macro avg       0.85      0.56      0.67      9782
weighted avg       0.84      0.54      0.65      9782

